Machine translation is a challenge for computers not to only understand human languages but also to generate languages. A machine translation can be viewed as a conditional language model, given a source sentence $x_i$, we needed to calculate the probability of generated sentence $p(y_i|x_i)$. In early years, statistical machine translation(SMT) was a focus, amongst which IBM models were basis, if you are interested, please visit Michael Collins' [webpage](http://www.cs.columbia.edu/~mcollins/), there he provided many useful and explicit lecture notes to illustrate basis terms and models of SMT.

In recent years, with the development of artificial neural networks as well as deep learning applications, neural translation models were explored, especially [seq2seq](https://arxiv.org/pdf/1406.1078v3.pdf) model as well as later models has improved performances of machine translation.

This project aims to realize a simple neural machine translation model through seq2seq concept, namely we only transform the source sentence into a fixed vevctor as context input for decoder.  More information, please read KyunghyunCho's article [Learning Phrase Representations using RNN Encoder–Decoder for Statistical Machine Translation](https://arxiv.org/pdf/1406.1078v3.pdf) .

Actually, there are some [excellent blogs](http://blog.csdn.net/u011414416/article/details/51048994) in Chinese, which introduced the development and theoretic models of neural machine translation explicitly and systemamtically.

Specify the paths of the original dataset.

In [3]:
# Data Parameters
data_dir = 'temp'
data_file = 'eng_ger.txt'

In [4]:
# Make data directory
import os
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

## Data Acquiry

Download the data from website if it does not exist.

In [5]:
class dataReader:
    '''
    Read text files from local drive.
    If not exists, download it.
    '''
    def __init__(self, file_path):
        self.file_path = file_path
        if not self.__checkExists():
            self.__download()
        
            
    def loadData(self):
        #print('Data Exists!')
        eng_ger_data = []
        with open(self.file_path, 'r') as in_conn:
            for row in in_conn:
                eng_ger_data.append(row[:-1])
        return eng_ger_data

    def __download(self):
        '''Download text files which contain translation pairs'''
        print('Data not found, downloading Eng-Ger sentences from www.manythings.org')
        sentence_url = 'http://www.manythings.org/anki/deu-eng.zip'
        r = urllib.request.urlopen(sentence_url)
        z = ZipFile(io.BytesIO(r.read()))
        file = z.read('deu.txt')
        # Format Data
        eng_ger_data = file.decode()
        eng_ger_data = eng_ger_data.encode('ascii',errors='ignore')
        eng_ger_data = eng_ger_data.decode().split('\n')
        # Write to file
        with open(self.file_path, 'w') as out_conn:
            for sentence in eng_ger_data:
                out_conn.write(sentence + '\n')
                
    def __checkExists(self):
        '''Check the file'''
        return os.path.isfile(self.file_path)

In [6]:
dr = dataReader('eng_ger.txt')
eng_ger_data = dr.loadData()

In [7]:
eng_ger_data[:10]

['Hi.\tHallo!',
 'Hi.\tGr Gott!',
 'Run!\tLauf!',
 'Wow!\tPotzdonner!',
 'Wow!\tDonnerwetter!',
 'Fire!\tFeuer!',
 'Help!\tHilfe!',
 'Help!\tZu Hlf!',
 'Stop!\tStopp!',
 'Wait!\tWarte!']

## Data Processig

Preprocess the original data.

In [66]:
import string
from collections import Counter
vocab_size = 10000
class textHandler:
    '''Split sentences into pairs of Source-Target language'''
    def __init__(self, data, vocab_size):
        self.data = data
        self.vocab_size = vocab_size
        self.__sentSplit()
        
    def __removePunctuation(self):
        '''Remove punctuation'''
        # Remove punctuation
        punct = string.punctuation
        pair_data = [''.join(char for char in sent if char not in punct) for sent in self.data]
        return pair_data
    
    def __sentSplit(self):
        # Break each sentence pair by tabs, one part is English, the other is German. 
        pair_data = self.__removePunctuation()
        s_t_data = [x.split('\t') for x in pair_data if len(x)>=1]
        [source_sentence, target_sentence] = [list(x) for x in zip(*s_t_data)]
        #Split each sentence into words
        self.source_sentence = [x.lower().split() for x in source_sentence]
        self.target_sentence = [x.lower().split() for x in target_sentence]
        #return source_sentence, target_sentence
    
    def __buildVocab(self, sents):
        '''Build Vocabulary for both languages'''
        # Process the English Vocabulary
        all_words = [word for sent in sents for word in sent]
        #Count the frequency of English words
        all_words_counts = Counter(all_words)
        #Get the most frequent vocab_size words, left regarded as unknow
        word_keys = [x[0] for x in all_words_counts.most_common(self.vocab_size-3)] 
        #Word to ID, set Starting token as 'SOS', ending token as 'EOS'
        vocab2ix = dict(zip(word_keys, range(2,self.vocab_size)))
        vocab2ix['SOS'] = 0
        vocab2ix['EOS'] = 1
        vocab2ix['UNK'] = self.vocab_size - 1
        #ID to Word
        ix2vocab = {val:key for key, val in vocab2ix.items()}
        return vocab2ix, ix2vocab
    
    def getSents(self):
        '''Get preprocessed sentences'''
        return self.source_sentence, self.target_sentence
    
    def sent2vec(self, sents, vocab2ix):
        '''Transform sentences into Ids'''
        processed = []
        for sent in sents:
            temp_sentence = []
            for word in sent:
                try:
                    temp_sentence.append(vocab2ix[word])
                except:
                    #Unknown words
                    temp_sentence.append(self.vocab_size-1)
            processed.append(temp_sentence)
        return processed
    
    def generateVocab(self):
        '''Generate Vocabulary'''
        #source_sentence, target_sentence = self.__sentSplit()
        source_vocab2ix, source_ix2vocab = self.__buildVocab(self.source_sentence)
        target_vocab2ix, target_ix2vocab = self.__buildVocab(self.target_sentence)
        return source_vocab2ix, source_ix2vocab, target_vocab2ix, target_ix2vocab
        


In [67]:
th = textHandler(data=eng_ger_data, vocab_size=vocab_size)

In [68]:
english_sentence, german_sentence = th.getSents()

In [69]:
eng_vocab2ix, eng_ix2vocab, ger_vocab2ix, ger_ix2vocab = th.generateVocab()

In [70]:
english_processed = th.sent2vec(english_sentence, eng_vocab2ix)
german_processed = th.sent2vec(german_sentence, ger_vocab2ix)

In [71]:
test_data = ['I love this dog', 'What a nice day', 'This is a book']
test_data = [x.lower().split() for x in test_data]
test_data = th.sent2vec(test_data, eng_vocab2ix)

In [72]:
test_data

[[5, 168, 17, 191], [24, 7, 392, 117], [17, 8, 7, 123]]

# Build a simple encoder-decoder architecture

In this demo, we use a simple encoder-decoder architecture to train and infer translations. We encode all the word vectors in source sentences into a fixed vector, then make the fixed vector as an input for the decoder.

In [15]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()

In [31]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        #Get embedding series of input words
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        #Compress the input vectors into RNN
        for i in range(self.n_layers):
            output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        #Create a initial zero hidden state
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        #result = result.cuda() if use_cuda else result
        return result

In [32]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        #Create embedding
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, input, hidden):
        #Transform input word id into embedding
        output = self.embedding(input).view(1, 1, -1)
        #Generate output through input and last hidden state
        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        #Create a initial zero hidden state
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        #result = result.cuda() if use_cuda else result
        return result

## Training Data
In order to understand the mechanism of neural machine translation, we wrap a pair of translation sentences each time instead of a batch of them.

In [33]:
hidden_size = 256
max_length = 10
encoder1 = EncoderRNN(vocab_size, hidden_size)

In [34]:
index = 100
input_variable, target_variable = english_processed[index], german_processed[index]

In [35]:
#Transform the input data and target into Variable vectors
input_variable = Variable(torch.LongTensor(input_variable).view(-1, 1))
target_variable = Variable(torch.LongTensor(target_variable).view(-1, 1))
input_length = input_variable.size()[0]
target_length = target_variable.size()[0]

### Encoder
We can compress a sereis of word embeddings into a final hidden state and output through RNN.
$$h_t = f(h_{t-1}, x_t)$$

In [36]:
encoder_hidden = encoder1.initHidden()
encoder_outputs = Variable(torch.zeros(max_length, encoder1.hidden_size))
#Calculate the final state of input words
for ei in range(input_length):
    encoder_output, encoder_hidden = encoder1(
        input_variable[ei], encoder_hidden)
    encoder_outputs[ei] = encoder_output[0][0]

### Decoder without Attention

In the decoder part,for training, we only take two inputs into consideration: The first is the target variables provided, and the second is the previous hidden state initialized by the final state($C_T$) of the encoder.
$$h_t = f(h_{t-1}, y_{t-1}), h_0=C_T$$

The architecture is as below(quoted from http://blog.csdn.net/jerr__y/article/details/53749693):
![encoder-decoder](http://img.blog.csdn.net/20170509152556448?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSmVycl9feQ==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

In [37]:
#Create an instance for decoder
decoder1 = DecoderRNN(hidden_size, vocab_size)

In [38]:
learning_rate = 0.001
encoder_optimizer = optim.SGD(encoder1.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder1.parameters(), lr=learning_rate)

In [39]:
loss = 0
criterion = nn.NLLLoss()
decoder_input = Variable(torch.LongTensor([[0]]))
#Set the beginning hidden state of decoder as the final state of encoder
decoder_hidden = encoder_hidden
for di in range(target_length):
    decoder_output, decoder_hidden = decoder1(
        decoder_input, decoder_hidden)
    loss += criterion(decoder_output[0], target_variable[di])
    #Set the target as input
    decoder_input = target_variable[di]  # Teacher forcing

In [40]:
print(loss)

Variable containing:
 18.3088
[torch.FloatTensor of size 1]



## Wrap it up

Now, we can put the training procedures in one function. Note, in the paper [Sequence to Sequence Learning with Neural Networks
](https://arxiv.org/pdf/1409.3215.pdf), they mentioned that it would be more efficient if we reversed the order of words in source sentences because the encoder could retain more information of the last few words.

In [41]:
import copy
compressed = list(zip(english_processed, german_processed))

In [42]:
sent_pairs = copy.deepcopy(compressed)

In [45]:
def appendTokens(s):
    '''Add ending tokens to each sentence'''
    s.append(1)
    return s

def reverseSent(sent):
    '''Reverse the order of words in a sentence'''
    sent = list(reversed(sent))
    return sent

In [46]:
import numpy as np
#Filter those long sentences
pairs_filtered = []
#Because we need to add one ending tokens later, so substract 1 here
np.random.shuffle(sent_pairs)
for item in sent_pairs[:10000]:
    if len(item[0]) <= (max_length-1) and len(item[0]) > 3:
        s, t = item[0], item[1]
        s = reverseSent(s)
        pairs_filtered.append((s, t))

In [47]:
criterion = nn.NLLLoss()
def training(encoder, decoder, encoder_optimizer, decoder_optimizer, epochs=1):
    for e in range(epochs):
        np.random.shuffle(pairs_filtered)
        for c, pair in enumerate(pairs_filtered):
            #Add ending tokens for each pair
            input_data, target_data = pair[0], pair[1]
            input_data.append(1)
            target_data.append(1)
            #Transform the input data and target into Variable vectors
            input_variable = Variable(torch.LongTensor(input_data).view(-1, 1))
            target_variable = Variable(torch.LongTensor(target_data).view(-1, 1))
            input_length = input_variable.size()[0]
            target_length = target_variable.size()[0]
            encoder_hidden = encoder.initHidden()
            encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
            #Calculate the final state of input words
            for i in range(input_length):
                encoder_output, encoder_hidden = encoder(
                    input_variable[i], encoder_hidden)
                if i == max_length:
                    print(c, pair)
                encoder_outputs[i] = encoder_output[0][0]
            #Clear grads
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            loss = 0
            decoder_input = Variable(torch.LongTensor([[0]]))
            #Set the beginning hidden state of decoder as the final state of encoder
            decoder_hidden = encoder_hidden
            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(
                    decoder_input, decoder_hidden)
                #print(decoder_output[0].size())
                #print('*'*20)
                #print(target_variable[di])
                loss += criterion(decoder_output[0], target_variable[di])
                #Set the target as input
                decoder_input = target_variable[di]  # Teacher forcing
            loss.backward()
            encoder_optimizer.step()
            decoder_optimizer.step()
            if c%200 == 0:
                print(loss.data[0] / target_length)

In [48]:
encoder1 = EncoderRNN(vocab_size, hidden_size)
decoder1 = DecoderRNN(hidden_size, vocab_size)
encoder1_optimizer = optim.SGD(encoder1.parameters(), lr=learning_rate)
decoder1_optimizer = optim.SGD(decoder1.parameters(), lr=learning_rate)
training(encoder1, decoder1, encoder1_optimizer, decoder1_optimizer)

9.28051503499349
8.087350845336914
7.310090065002441
6.489105987548828
6.605953216552734
6.173790613810222
7.359162466866629
6.489985874720982
5.146819591522217
6.609954410129124
5.694840567452567
7.703678894042969
7.36232328414917
5.88268062046596
5.424562454223633
6.668548107147217
7.05186398824056
7.121327877044678
5.375429630279541
6.207205295562744
5.655857563018799
6.4616539001464846
5.991965611775716
6.0854900905064175
6.7501373291015625
4.970180892944336
4.9261124928792315
6.565703392028809
5.912744522094727
5.052161625453404
7.354418667879972
5.72467041015625
4.825438499450684
6.486326217651367
5.998085021972656
5.692766462053571
4.788918813069661
5.158510843912761
4.944438298543294
4.610002899169922
4.6514303419325085


And next, we use a greedy method to generate a target sentence based on source sentence. Each time, we selected the word wich has the maximum probability untile the decoder generate an ending token 'EOS'. However, a best choice each time does not guarantte the most-likely sentence in the end.

In [49]:
def evaluate_greedy(encoder, decoder, sentence, max_length=10):
    input_variable = Variable(torch.LongTensor(sentence).view(-1, 1))
    input_length = input_variable.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    #encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei],
                                                 encoder_hidden)
        #encoder_outputs[ei] = encoder_outputs[ei] + encoder_output[0][0]
    
    #Set the inital value as SOS token
    decoder_input = Variable(torch.LongTensor([[0]]))  # SOS

    decoder_hidden = encoder_hidden
    decoded_words = []

    #Greedy method
    for di in range(max_length):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        top_value, top_index = decoder_output.data.topk(1)
        #Get the index of the word
        ni = top_index[0][0]
        if ni == 1:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(ger_ix2vocab[ni])

        decoder_input = Variable(torch.LongTensor([[ni]]))
        #decoder_input = decoder_input.cuda() if use_cuda else decoder_input


    print(decoded_words)

In [79]:
def beamsearch_topk(decoder, decoder_input, decoder_hidden, k=2):
    '''Beam Search Method'''
    decoder_output, decoder_hidden = decoder(
        decoder_input, decoder_hidden)
    top_value, top_index = decoder_output.data.topk(k)
    #first, second = top_index[0][0], top_index[0][1]
    return top_value, top_index, decoder_hidden

In [102]:
def evaluate_beamsearch(encoder, decoder, sentence, max_length=10):
    input_variable = Variable(torch.LongTensor(sentence).view(-1, 1))
    input_length = input_variable.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    #encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei],
                                                 encoder_hidden)
        #encoder_outputs[ei] = encoder_outputs[ei] + encoder_output[0][0]
    
    #Set the inital value as SOS token
    decoder_input = Variable(torch.LongTensor([[0]]))  # SOS

    decoder_hidden = encoder_hidden
    

    
    #Beam search method
    #Get the largest k values and their indice
    k = 2
    top_value, top_index, decoder_hidden = beamsearch_topk(decoder, decoder_input, decoder_hidden)
    log_sum = top_value[0]
    current_index = top_index[0]
    #Create a recorder to record path
    record_path_dict = {}
    for i in range(k):
        record_path_dict[i] = []
    
    #recordd hidden state
    decoder_hidden_dict = {}
    for i in range(k):
        decoder_hidden_dict[i] = decoder_hidden
    for di in range(1, max_length):
        previous_index = current_index
        #Traverse each selected word
        log_sum_list = []
        k_top_list = []
        for i, ix in enumerate(previous_index):
            input_id = Variable(torch.LongTensor([[ix]]))
            top_value_temp, top_index_temp, decoder_hidden_temp = beamsearch_topk(decoder, 
                                                                                  input_id, decoder_hidden_dict[i])
            for j, v in enumerate(top_value_temp[0]):
                temp_sum = log_sum[i] + v
                #Record sums
                log_sum_list.append(temp_sum)
                #Record pairs
                k_top_list.append([ix, top_index_temp[0][j], decoder_hidden_temp])
                
        sum_ix_pair = list(zip(log_sum_list, k_top_list))
        sum_ix_pair.sort(reverse=True)
        for i in range(k):
            #Update selected word ID and log sums of probability
            current_index[i] = sum_ix_pair[i][1][1]
            log_sum[i] += sum_ix_pair[i][0]
            #Record the path
            record_path_dict[i].append(sum_ix_pair[i][1][:2])
            decoder_hidden_dict[i] = sum_ix_pair[i][1][2]

        #first, second = top_index[0][0], top_index[0][1]
        #if first == 1:
            #decoded_words.append('<EOS>')
            #break
        #else:
    for i in range(k):
        decoded_words = []
        print('The ' + str(i) + ' word chain:')
        for ix_pair in record_path_dict[i]:
            decoded_words.append(ger_ix2vocab[ix_pair[0]])
            decoded_words.append(ger_ix2vocab[ix_pair[1]])
            if ix_pair[1] == 1:
                break
            #decoded_words.append(ger_ix2vocab[first])
        print(decoded_words)

        #decoder_input = Variable(torch.LongTensor([[first]]))

    

In [103]:
test_data[1]

[24, 7, 392, 117]

In [104]:
evaluate_greedy(encoder1, decoder1, test_data[1])

['sie', 'ist', 'UNK', '<EOS>']


In [105]:
evaluate_beamsearch(encoder1, decoder1, test_data[1])

The 0 word chain:
['ich', 'habe', 'habe', 'UNK', 'UNK', 'EOS']
The 1 word chain:
['sie', 'ist', 'habe', 'ein', 'UNK', 'UNK', 'UNK', 'EOS']
